# Reconnaissance: can numba-scfg handle mcl

In this notebook we explore the extent to which the numba-scfg package can handle the array methods in the mcl package.

In [5]:
# simple demo pipeline
import ast
from numba_scfg.core.datastructures.ast_transforms import (
    AST2SCFGTransformer,
    SCFG2ASTTransformer,
    unparse_code,
)


def func() -> int:
    c = 0
    for i in range(10):
        c += i
        for j in range(10):
            c += j
    return c


scfg = AST2SCFGTransformer(func).transform_to_SCFG()
scfg.restructure()

original_ast = unparse_code(func)[0]
transformed_ast = SCFG2ASTTransformer().transform(
    original=original_ast, scfg=scfg)
transformed_source = ast.unparse(transformed_ast)
print(transformed_source)

def transformed_func() -> int:
    c = 0
    __scfg_iterator_1__ = iter(range(10))
    i = None
    __scfg_loop_cont_1__ = True
    while __scfg_loop_cont_1__:
        __scfg_iter_last_1__ = i
        i = next(__scfg_iterator_1__, '__scfg_sentinel__')
        if i != '__scfg_sentinel__':
            c += i
            __scfg_iterator_5__ = iter(range(10))
            j = None
            __scfg_loop_cont_2__ = True
            while __scfg_loop_cont_2__:
                __scfg_iter_last_5__ = j
                j = next(__scfg_iterator_5__, '__scfg_sentinel__')
                if j != '__scfg_sentinel__':
                    c += j
                    __scfg_backedge_var_1__ = 0
                else:
                    __scfg_backedge_var_1__ = 1
                __scfg_loop_cont_2__ = not __scfg_backedge_var_1__
            j = __scfg_iter_last_5__
            __scfg_backedge_var_0__ = 0
        else:
            __scfg_backedge_var_0__ = 1
        __scfg_loop_cont_1__ = not __scfg_bac

In [12]:
def trans(func):
    scfg = AST2SCFGTransformer(func).transform_to_SCFG()
    scfg.restructure()

    original_ast = unparse_code(func)[0]
    transformed_ast = SCFG2ASTTransformer().transform(
        original=original_ast, scfg=scfg)
    transformed_source = ast.unparse(transformed_ast)
    print(transformed_source)

In [13]:
# def __setitem__(self, idx: _Indices, value: T):
def __setitem__(self, idx, value):
    if not isinstance(idx, tuple):
        idx = (idx,)

    if len(idx) < len(self.shape):
        idx = idx + (slice(None),) * (len(self.shape) - len(idx))

    if any(isinstance(i, Array) for i in idx):
        #raise ValueError("Array indexing is not supported in setitem")
        pass
    elif any(isinstance(i, slice) for i in idx):
        array_view = self.__getitem__(idx)
        if isinstance(value, Array):
            if array_view.shape != value.shape:
                #raise ValueError("Shapes do not match")
                pass
            for idx_ in LoopNestAPI.from_tuple(array_view.shape):
                array_view[idx_] = value[idx_]
        else:
            if isinstance(value, Integer):
                value = value.value
            for idx in LoopNestAPI.from_tuple(array_view.shape):
                array_view[idx] = value
    else:
        idx = tuple_cast(intp, idx)
        # TODO: There's no assertion that checks if idx is within bounds.
        if isinstance(value, Integer):
            value = value.value
        self.data.store(idx, value)

trans(__setitem__)

AttributeError: 'UnaryOp' object has no attribute 'value'

In [18]:
# def __getitem__(self, idx: _Indices) -> Array[T] | Generic:
def __getitem__(self, idx):
    if not isinstance(idx, tuple):
        idx = (idx,)

    if len(idx) < len(self.shape):
        idx = idx + (slice(None),) * (len(self.shape) - len(idx))

    if self.is_advanced(idx):
        res_shape, subspace_shape, subspace_offset = self.fancy_shape(idx)
        new_memref = memref.alloc(res_shape, i32)
        res_array = Array(dtype=self.dtype, data=new_memref)

        for _idx in LoopNestAPI.from_tuple(subspace_shape):
            res_idx = tuple([slice(None)] * subspace_offset + list(_idx))
            array_idx = tuple([i if isinstance(i, (int, slice)) else intp(i[_idx].value) for i in idx])
            res_array[*res_idx] = self[*array_idx]
        return res_array
    elif any(isinstance(i, slice) for i in idx):
        if any(isinstance(i, Array) for i in idx):
            #raise ValueError("Singular array indexing is not supported")
            pass
        new_shape, new_strides, new_offset = self.new_arrayinfo(idx)
        new_memref = self.data.view(new_shape, new_strides, new_offset)
        return Array(dtype=self.dtype, data=new_memref)
    else:
        # All indices are integers
        idx = tuple_cast(intp, idx)
        # TODO: There's no assertion that checks if idx is within bounds.
        return self.dtype.type.from_memory(self.data, idx)

trans(__getitem__)

AssertionError: 

In [24]:
# def broadcast_to(self, shape: tuple[intp, ...]) -> None:
def broadcast_to(self, shape):
    # This function can also serve as a assertion
    new_shape = self.broadcast_shapes(self.shape, shape)
    #assert new_shape == shape

    old_shape = self.shape
    old_strides = self.strides
    new_ndim = len(new_shape)
    ndim_diff = len(new_shape) - len(old_shape)
    extended_shape = (intp(1),) * ndim_diff + old_shape
    extended_strides = (intp(0),) * ndim_diff + old_strides
    new_strides = [intp(0)] * new_ndim
    for i in range(new_ndim):
        if new_shape[i] == extended_shape[i]:
            new_strides[i] = extended_strides[i] * (new_shape[i] // extended_shape[i])

    new_strides = tuple(new_strides)

    self.data = self.data.view(new_shape, new_strides, self.data.offset)

trans(broadcast_to)

def transformed_broadcast_to(self, shape):
    new_shape = self.broadcast_shapes(self.shape, shape)
    old_shape = self.shape
    old_strides = self.strides
    new_ndim = len(new_shape)
    ndim_diff = len(new_shape) - len(old_shape)
    extended_shape = (intp(1),) * ndim_diff + old_shape
    extended_strides = (intp(0),) * ndim_diff + old_strides
    new_strides = [intp(0)] * new_ndim
    __scfg_iterator_1__ = iter(range(new_ndim))
    i = None
    __scfg_loop_cont_1__ = True
    while __scfg_loop_cont_1__:
        __scfg_iter_last_1__ = i
        i = next(__scfg_iterator_1__, '__scfg_sentinel__')
        if i != '__scfg_sentinel__':
            if new_shape[i] == extended_shape[i]:
                new_strides[i] = extended_strides[i] * (new_shape[i] // extended_shape[i])
                __scfg_backedge_var_0__ = 0
            else:
                __scfg_backedge_var_0__ = 0
        else:
            __scfg_backedge_var_0__ = 1
        __scfg_loop_cont_1__ = not __scfg_backedge_var

In [25]:
#@classmethod
#def is_advanced(cls, idx: _Indices) -> bool:
def is_advanced(cls, idx):
    return any((isinstance(i, Array) and i.ndim > intp(0)) for i in idx)
trans(is_advanced)

def transformed_is_advanced(cls, idx):
    return any((isinstance(i, Array) and i.ndim > intp(0) for i in idx))


In [30]:
#@classmethod
#def broadcast_shapes(cls, *shapes) -> tuple[intp, ...]:
def broadcast_shapes(cls, *shapes):
    if len(shapes) == 0:
        #raise ValueError("At least one shape is required")
        pass
    
    max_ndim = max(map(len, shapes))
    identity = intp(1)

    # The length can be determined at compile time
    # So something like tuple setitem can be used
    # here instead of building a list.
    result = [identity] * max_ndim

    for i in range(1, max_ndim + 1):
        curr_axis_shape = identity
        for shape in shapes:
            if len(shape) + 1 > i:
                if shape[-i] == identity:
                    pass
                elif curr_axis_shape is identity:
                    curr_axis_shape = shape[-i]
                elif curr_axis_shape != shape[-i]:
                    #raise ValueError("Shapes are not broadcastable")
                    pass
        result[-i] = curr_axis_shape

    return tuple(result)
trans(broadcast_shapes)

AssertionError: 

In [34]:
#def new_arrayinfo(self, idx: _Indices) -> tuple[intp, ...]:
def new_arrayinfo(self, idx):
    res_ndim = self.ndim

    for idx_ in idx:
        if isinstance(idx_, (int, intp)):
            res_ndim -= intp(1)

    res_shape = [intp(0)] * res_ndim
    res_strides = [intp(0)] * res_ndim
    res_offset = intp(0)

    curr_idx = 0
    for i, idx_ in enumerate(idx):
        if isinstance(idx_, slice):
            if idx_.step is not None:
                #raise ValueError("Slicing with step is not supported")
                pass
            
            if idx_.start is not None and idx_.start < 0:
                #raise ValueError("Negative slicing is not supported")
                pass
            if idx_.stop is not None and idx_.stop < 0:
                #raise ValueError("Negative slicing is not supported")
                pass
            if idx_.start is None:
                idx_start = intp(0)
            else:
                idx_start = intp(idx_.start)

            if idx_.stop is None:
                idx_stop = self.shape[i]
            else:
                idx_stop = intp(idx_.stop)
            res_shape[curr_idx] = idx_stop - idx_start
            res_strides[curr_idx] = self.strides[i]
            res_offset += idx_start * self.strides[i]
            curr_idx += 1
        elif isinstance(idx_, (int, intp)):
            res_offset += intp(idx_) * self.strides[i]

    return tuple(res_shape), tuple(res_strides), res_offset
trans(new_arrayinfo)

AssertionError: 

In [37]:
#def fancy_shape(self, idx: _Indices) -> tuple[intp, ...]:
def fancy_shape(self, idx):
    res_ndim = self.ndim
    subspace_offset = 0
    num_subspaces = 0
    in_subspace = False
    num_slices = 0
    array_shapes = []
    slice_shapes = []

    for i, idx_ in enumerate(idx):
        if isinstance(idx_, (int, intp)):
            res_ndim -= intp(1)
            in_subspace = False
            if subspace_offset <= 0:
                subspace_offset -= 1
        elif isinstance(idx_, Array):
            if not in_subspace:
                in_subspace = True
                num_subspaces += 1
            if subspace_offset <= 0:
                subspace_offset += i
            res_ndim -= intp(1)
            array_shapes.append(idx_.shape)
        elif isinstance(idx_, slice):
            num_slices += 1
            in_subspace = False
            if idx_.start is None:
                idx_start = intp(0)
            else:
                idx_start = intp(idx_.start)
            if idx_.stop is None:
                idx_stop = self.shape[i]
            else:
                idx_stop = intp(idx_.stop)
            slice_shapes.append(idx_stop - idx_start)
        else:
            #raise ValueError("Invalid index")
            pass

    if num_subspaces > 1:
        subspace_offset = 0

    subspace_shape = self.broadcast_shapes(*array_shapes)

    res_shape = slice_shapes[:subspace_offset] + list(subspace_shape) + slice_shapes[subspace_offset:]

    return tuple(res_shape), tuple(subspace_shape), subspace_offset

trans(fancy_shape)

AttributeError: 'Call' object has no attribute 'value'

## Summary:

* type annotations not supported in isolation
* `raise` not supported
* `assert` not supported
* `AttributeError: 'UnaryOp' object has no attribute 'value` assume `UnaryOp` not supported
* generators (maybe also comprhensions) pass through unchanged
* Some `AssertionError`s related to multiple exit blocks
* Some `AssertionError`s related to multiple heads
* `AttributeError: 'Call' object has no attribute 'value'`